globale inportes and veriables

In [57]:
import cv2
import os
import time


import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/kianf/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'
from PIL import Image
import os
import numpy as np


grid_size = 40  # Grid size
move_up = 30  # Number of times to move up
wait_time = 1000  # Wait time in milliseconds
increment = 2  # Distance between each move


data_1d = np.empty(((grid_size * grid_size * move_up) // increment), dtype=object)  # Create a 1D array to hold strings

# Create a 3D array with dimensions (move_up, grid_size, grid_size)
Data_3d = np.zeros((int(move_up/increment), int(grid_size/increment), int(grid_size/increment)))





Image tacking functions

In [59]:


def setup_camera():
    # Create a VideoCapture object
    cap = cv2.VideoCapture(0)
    
    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        cap = None
        
    return cap


def take_image(cap, image_num):    
    # Capture a single frame
    ret, frame = cap.read()

    if ret:
        # Create the directory if it doesn't exist
        if not os.path.exists('images'):
            os.makedirs('images')

        # Save the captured frame to the 'images' directory
        cv2.imwrite(os.path.join('images', 'captured_image_'+str(image_num)+'.jpg'), frame)
        print(f"Image saved to 'captured_image_{image_num}.jpg'.")
    else:
        print("Error: Could not read frame.")
        frame = None
    
    return frame


def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

def crop_image(image, x, y, w, h):
    cropped = image[y:y+h, x:x+w]
    return cropped

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def convert_to_grayscale(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grayscale
    
    
    # Load the first captured image
image_path = os.path.join('images', 'captured_image_0.jpg')
image = cv2.imread(image_path)

def print_pixel_values(image):
    """
    Print the pixel values of a black and white image.
    
    Parameters:
    image (numpy.ndarray): The black and white image.
    """
    for row in image:
        for pixel in row:
            print(pixel, end=' ')
        print()

def threshold_image(image, threshold_value):
    
    # Apply the threshold
    _, thresholded = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded

def blur_image(image):
    blurred = cv2.GaussianBlur(image, (1, 17), 0)
    return blurred


# Check if the image was loaded successfully
if image is not None:
    # Define the crop parameters (x, y, width, height)
    print("Image loaded successfully.")
    x, y, w, h = 260, 50, 260, 70
    cropped_image = crop_image(image, x, y, w, h)
    # sharpened_image = sharpen_image(cropped_image)
    black_and_white = convert_to_grayscale(cropped_image)

    blur_image = blur_image(black_and_white)
    
    
    threshold_value = 130
    thresholded_image = threshold_image(black_and_white, threshold_value)



        
    # Save the cropped image
    cropped_image_path = os.path.join('images', 'cropped_image_0.jpg')
    cv2.imwrite(cropped_image_path, cropped_image)
    print(f"Cropped image saved to 'cropped_image_0.jpg'.")

    

    # Save the black and white image
    black_and_white_path = os.path.join('images', 'black_and_white_0.jpg')
    cv2.imwrite(black_and_white_path, black_and_white)
    print(f"Black and white image saved to 'black_and_white_0.jpg'.")

    blur_image_path = os.path.join('images', 'blur_image_0.jpg')
    cv2.imwrite(blur_image_path, blur_image)
    print(f"Blur image saved to 'blur_image_0.jpg'.")

        # Save the thresholded image
    thresholded_image_path = os.path.join('images', 'thresholded_image_0.jpg')
    cv2.imwrite(thresholded_image_path, thresholded_image)
    print(f"Thresholded image saved to 'thresholded_image_0.jpg'.")    
else:
    print("Error: Could not load image.")




display(image_to_text(cropped_image_path))
display(image_to_text(black_and_white_path))
display(image_to_text(blur_image_path))
display(image_to_text(thresholded_image_path))




Image loaded successfully.
Cropped image saved to 'cropped_image_0.jpg'.
Black and white image saved to 'black_and_white_0.jpg'.
Blur image saved to 'blur_image_0.jpg'.
Thresholded image saved to 'thresholded_image_0.jpg'.


'45.7 '

'4. '

'9. '

''

image to text functions

In [50]:


def image_to_text(image_path):
    """
    Extract text from an image using Tesseract OCR.
    
    Parameters:
    image_path (str): The path to the image file.
    
    Returns:
    str: The extracted text.
    """
    # Open the image file
    img = Image.open(image_path)
    
    # Use pytesseract to do OCR on the image
    
    text = pytesseract.image_to_string(img, config='--psm 7 -c tessedit_char_whitelist=0123456789.- ')
    text = text.replace('\n', ' ')
    return text

def get_data(image_num):
    image_path = os.path.join('images', 'captured_image_'+str(image_num)+'.jpg')
    extracted_text = image_to_text(image_path)
    return extracted_text


fill array function

In [51]:
def fill_data2D():
    for i in range(0, len(data_1d)):
        image_num = i
        extracted_text = get_data(image_num)
        data_1d[i] = extracted_text
        print(data_1d[i])




set up camrer

In [52]:
num_images = 5  # Number of images to capture
cap = setup_camera()

take image every set time interval

In [53]:
# def take_images_every_second(cap, num_images):
#     start_time = time.time()
#     for i in range(num_images):
#         frame = take_image(cap, i)
#         data_1d[i-1]= get_data(i)
#         end_time= time.time()
#         elapsed_time = end_time - start_time
#         sleep_time = max(0, 1 - elapsed_time)
#         time.sleep(sleep_time)
#         print(f" time: {elapsed_time:.2f} seconds")
#         start_time = time.time()  # Reset start time for the next iteration

# # Example usage




# take_images_every_second(cap, num_images)


print resaltes